# **JPM MLCOE TSRL 2026 Q1**
---
**Heartie CHEN**

## 0. Global Configurations
---

In [1]:
# Core dependencies for data ingestion, modeling, and evaluation.
from pathlib import Path
import json
import re
import random
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from typing import Tuple

pd.set_option('display.float_format', lambda v: f"{v:,.2f}")


print('yfinance:', yf.__version__)
print('pandas:', pd.__version__)
print('tensorflow:', tf.__version__)

yfinance: 1.1.0
pandas: 3.0.0
tensorflow: 2.20.0


In [4]:
# Fix RNG seeds for reproducibility across runs.
CONFIG_DIR = Path('..').resolve() / 'config'
with open(CONFIG_DIR / 'model_config.json', 'r', encoding = 'utf-8-sig') as f:
    MODEL_CONFIG = json.load(f)

SEED = int(MODEL_CONFIG['seed'])
random.seed(SEED)
np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

# Configure cache location and statement frequency.
DATA_DIR = Path('..').resolve() / MODEL_CONFIG['data_dir']
DATA_DIR.mkdir(parents = True, exist_ok = True)
TARGET_FREQ = MODEL_CONFIG['target_freq']

## 1. Data Pipeline
---

### 1.1 Column Aliases and Identity Checks

In [5]:
# Provider column aliases to canonical balance sheet names.
BS_ALIASES = MODEL_CONFIG['bs_aliases']


def canonicalize_bs(df: pd.DataFrame) -> pd.DataFrame:
    """Return a balance sheet with canonical column names."""

    rename_map = {}

    for canon, candidates in BS_ALIASES.items():
        for c in candidates:

            if c in df.columns:
                rename_map[c] = canon
                
                break


    return df.rename(columns = rename_map)


def identity_residual(df: pd.DataFrame) -> pd.Series:
    """Compute balance sheet identity residuals."""

    if "Total Liabilities" not in df.columns or "Total Equity" not in df.columns:
        df = canonicalize_bs(df)
    
    required = ["Total Assets", "Total Liabilities", "Total Equity"]
    missing = [c for c in required if c not in df]

    if missing:

        raise KeyError(f"Missing columns: {missing}")


    return df["Total Assets"] - (df["Total Liabilities"] + df["Total Equity"])


def summarize_identity(resid: pd.Series) -> pd.Series:
    """Summarize residual statistics."""

    return pd.Series({

        "mean": resid.mean(),
        "std": resid.std(),
        "max_abs": resid.abs().max(),
    })

### 1.2 Load Data with Caching

In [6]:
# Ticker normalization and cached statement loading utilities.
SPECIAL_TICKERS = MODEL_CONFIG['special_tickers']
FISCAL_YEAR_END_MONTHS_RAW = MODEL_CONFIG.get('fiscal_year_end_months', {})


def normalize_ticker(raw: str) -> str:
    """Normalize tickers and apply project-specific mappings."""

    value = raw.strip()
    upper = value.upper()

    if upper in SPECIAL_TICKERS:

        return SPECIAL_TICKERS[upper]

    if re.fullmatch(r"\d+", value):

        return value.zfill(4) + ".HK"


    return upper


FISCAL_YEAR_END_MONTHS = {
    normalize_ticker(k): int(v) for k, v in FISCAL_YEAR_END_MONTHS_RAW.items()
}


def slugify(ticker: str) -> str:
    """Create a filesystem-friendly slug for a ticker."""

    return re.sub(r"[^a-z0-9]+", "_", ticker.lower()).strip("_")


def _fetch_statements(tkr, freq: str):
    """Fetch statements from Yahoo Finance for a ticker."""

    bs = tkr.get_balance_sheet(freq = freq)
    is_df = tkr.get_financials(freq = freq)

    if bs is None or is_df is None:

        return pd.DataFrame(), pd.DataFrame()

    bs = bs.T.sort_index()
    is_df = is_df.T.sort_index()


    return bs, is_df


def _load_cached(path: Path) -> pd.DataFrame:
    """Load a cached statement CSV if available."""

    return pd.read_csv(path, index_col = 0, parse_dates = True)


def load_statements(ticker: str, freq: str = TARGET_FREQ):
    """Load or fetch statements for a ticker and frequency."""

    ticker = normalize_ticker(ticker)
    slug = slugify(ticker)
    freq = freq.lower()

    data_dir = DATA_DIR / freq
    data_dir.mkdir(parents = True, exist_ok = True)

    bs_path = data_dir / f"{slug}_balance_sheet_{freq}.csv"
    is_path = data_dir / f"{slug}_income_statement_{freq}.csv"

    # Prefer cached CSVs to reduce API calls.

    if bs_path.exists() and is_path.exists():
        bs = _load_cached(bs_path)
        is_df = _load_cached(is_path)

    else:
        tkr = yf.Ticker(ticker)
        bs, is_df = _fetch_statements(tkr, freq)

        if not bs.empty and not is_df.empty:
            bs.to_csv(bs_path)
            is_df.to_csv(is_path)

    if bs.empty or is_df.empty:

        raise RuntimeError(f"Failed to fetch statements for {ticker} ({freq})")

    bs = canonicalize_bs(bs)


    return bs, is_df, freq


TICKERS = MODEL_CONFIG['tickers']
PRIMARY_TICKER = MODEL_CONFIG.get('primary_ticker', TICKERS[0])
bs, is_df, stmt_freq = load_statements(PRIMARY_TICKER, freq = TARGET_FREQ)


print("Loaded tickers:", [normalize_ticker(t) for t in TICKERS])
print("Statement frequency:", stmt_freq)
print("Balance sheet shape:", bs.shape)
print("Income statement shape:", is_df.shape)
print("Identity residual stats:", summarize_identity(identity_residual(bs)))


bs.head()

Loaded tickers: ['AAPL', 'GOOG', '0700.HK', '1810.HK', 'IBM', 'TSLA', '9633.HK', '9987.HK', '9988.HK', 'IBKR', 'KO', 'MCD', 'EL', 'BRK-B', 'NESN.SW']
Statement frequency: yearly
Balance sheet shape: (5, 69)
Income statement shape: (5, 39)
Identity residual stats: mean      0.00
std       0.00
max_abs   0.00
dtype: float64


,TreasurySharesNumber,OrdinarySharesNumber,ShareIssued,NetDebt,TotalDebt,TangibleBookValue,InvestedCapital,WorkingCapital,NetTangibleAssets,CapitalLeaseObligations,...,OtherCurrentAssets,Inventory,Receivables,OtherReceivables,Accounts Receivable,CashCashEquivalentsAndShortTermInvestments,OtherShortTermInvestments,CashAndCashEquivalents,CashEquivalents,CashFinancial
2021-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"11,803,000,000.00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-30,NaN,"15,943,425,000.00","15,943,425,000.00","96,423,000,000.00","132,480,000,000.00","50,672,000,000.00","170,741,000,000.00","-18,577,000,000.00","50,672,000,000.00","12,411,000,000.00",...,"21,223,000,000.00","4,946,000,000.00","60,932,000,000.00","32,748,000,000.00","28,184,000,000.00","48,304,000,000.00","24,658,000,000.00","23,646,000,000.00","5,100,000,000.00","18,546,000,000.00"
2023-09-30,0.00,"15,550,061,000.00","15,550,061,000.00","81,123,000,000.00","111,088,000,000.00","62,146,000,000.00","173,234,000,000.00","-1,742,000,000.00","62,146,000,000.00","12,842,000,000.00",...,"14,695,000,000.00","6,331,000,000.00","60,985,000,000.00","31,477,000,000.00","29,508,000,000.00","61,555,000,000.00","31,590,000,000.00","29,965,000,000.00","1,606,000,000.00","28,359,000,000.00"
2024-09-30,NaN,"15,116,786,000.00","15,116,786,000.00","76,686,000,000.00","106,629,000,000.00","56,950,000,000.00","163,579,000,000.00","-23,405,000,000.00","56,950,000,000.00",NaN,...,"14,287,000,000.00","7,286,000,000.00","66,243,000,000.00","32,833,000,000.00","33,410,000,000.00","65,171,000,000.00","35,228,000,000.00","29,943,000,000.00","2,744,000,000.00","27,199,000,000.00"
2025-09-30,NaN,"14,773,260,000.00","14,773,260,000.00","62,723,000,000.00","98,657,000,000.00","73,733,000,000.00","172,390,000,000.00","-17,674,000,000.00","73,733,000,000.00",NaN,...,"14,585,000,000.00","5,718,000,000.00","72,957,000,000.00","33,180,000,000.00","39,777,000,000.00","54,697,000,000.00","18,763,000,000.00","35,934,000,000.00","7,667,000,000.00","28,267,000,000.00"


## 2. Features & Dataset

### 2.1 Feature Engineering

#### 2.1.1 Derived Drivers (DSO/DPO/DIH, Margins, Growth, Logs)

In [6]:
def _pick(df: pd.DataFrame, options):
    """Pick the first available series from the provided options."""

    for c in options:

        if c in df.columns:

            return df[c]

    raise KeyError(f'Missing columns: {options}')


def compute_features(bs: pd.DataFrame, is_df: pd.DataFrame, days: float = 365.0, growth_periods: int = 1) -> pd.DataFrame:
    """Compute financial ratios and growth features."""

    rev = _pick(is_df, ['Total Revenue', 'Operating Revenue', 'TotalRevenue', 'OperatingRevenue', 'Revenues'])
    cogs = _pick(is_df, ['Cost Of Revenue', 'Cost of Revenue', 'CostOfRevenue', 'ReconciledCostOfRevenue'])
    op_inc = _pick(is_df, ['Operating Income', 'OperatingIncome'])
    net_inc = _pick(is_df, ['Net Income', 'NetIncome', 'NetIncomeFromContinuingOperationNetMinorityInterest'])
    ar = _pick(bs, ['Accounts Receivable', 'AccountsReceivable'])
    ap = _pick(bs, ['Accounts Payable', 'AccountsPayable'])
    inv = _pick(bs, ['Inventory', 'Inventories'])

    feats = pd.DataFrame(index = bs.index)

    sales_per_day = rev / days
    cogs_per_day = cogs / days

    feats['dso'] = ar / sales_per_day
    feats['dpo'] = ap / cogs_per_day
    feats['dih'] = inv / cogs_per_day
    feats['gross_margin'] = (rev - cogs) / rev
    feats['op_margin'] = op_inc / rev
    feats['net_margin'] = net_inc / rev
    feats['rev_yoy'] = rev.pct_change(periods = growth_periods)
    feats['cogs_yoy'] = cogs.pct_change(periods = growth_periods)
    feats['netinc_yoy'] = net_inc.pct_change(periods = growth_periods)
    feats['log_rev'] = np.log1p(rev)
    feats['log_assets'] = np.log1p(_pick(bs, ['Total Assets', 'TotalAssets']))
    
    feats = feats.replace([np.inf, -np.inf], np.nan)
    feats = feats.sort_index().ffill().bfill()


    return feats


period_days = 365.0 if TARGET_FREQ == 'yearly' else 90.0
growth_periods = 1 if TARGET_FREQ == 'yearly' else 4

features = compute_features(bs, is_df, days = period_days, growth_periods = growth_periods)
features.tail()

,dso,dpo,dih,gross_margin,op_margin,net_margin,rev_yoy,cogs_yoy,netinc_yoy,log_rev,log_assets
2021-09-30,26.09,104.69,8.08,0.43,0.30,0.25,-0.03,-0.04,-0.03,26.70,26.59
2022-09-30,26.09,104.69,8.08,0.43,0.30,0.25,-0.03,-0.04,-0.03,26.70,26.59
2023-09-30,28.10,106.72,10.79,0.44,0.30,0.25,-0.03,-0.04,-0.03,26.67,26.59
2024-09-30,31.19,119.66,12.64,0.46,0.32,0.24,0.02,-0.02,-0.03,26.69,26.62
2025-09-30,34.89,115.40,9.45,0.47,0.32,0.27,0.06,0.05,0.19,26.75,26.61


### 2.2 Dataset Assembly

In [7]:
TARGET_BS = ['Total Assets', 'Total Liabilities', 'Total Equity']
TARGET_LE = ['Total Liabilities', 'Total Equity']
NET_INCOME_COL = 'Net Income'
STATE_COLS = [
    'Accounts Receivable', 'Accounts Payable', 'Inventory', 'Net PPE',
    'Total Liabilities', 'Total Equity', 'Retained Earnings'
]


def _has_any(df: pd.DataFrame, options: list[str]) -> bool:
    """Return True if any candidate column exists."""

    return any(c in df.columns for c in options)


def missing_required_columns(bs: pd.DataFrame, is_df: pd.DataFrame) -> list[str]:
    """Return missing required columns for BS and IS."""

    missing = []

    if not _has_any(bs, ['Total Assets', 'TotalAssets']):
        missing.append('Total Assets')
    
    if not _has_any(bs, ['Total Liabilities', 'Total Liabilities', 'TotalLiabilities', 'Total Liabilities Net Minority Interest', 'TotalLiabilitiesNetMinorityInterest']):
        missing.append('Total Liabilities')
    
    if not _has_any(bs, ['Total Equity', 'Stockholders Equity', 'StockholdersEquity', 'Total Equity Gross Minority Interest', 'TotalEquityGrossMinorityInterest']):
        missing.append('Total Equity')
    
    if not _has_any(bs, ['Accounts Receivable', 'AccountsReceivable']):
        missing.append('Accounts Receivable')
    
    if not _has_any(bs, ['Accounts Payable', 'AccountsPayable']):
        missing.append('Accounts Payable')
    
    if not _has_any(bs, ['Inventory', 'Inventories']):
        missing.append('Inventory')
    
    if not _has_any(bs, ['Net PPE', 'NetPPE']):
        missing.append('Net PPE')
    
    if not _has_any(bs, ['Retained Earnings', 'RetainedEarnings']):
        missing.append('Retained Earnings')

    if not _has_any(is_df, ['Total Revenue', 'Operating Revenue', 'TotalRevenue', 'OperatingRevenue', 'Revenues']):
        missing.append('Total Revenue')
   
    if not _has_any(is_df, ['Cost Of Revenue', 'Cost of Revenue', 'CostOfRevenue', 'ReconciledCostOfRevenue']):
        missing.append('Cost Of Revenue')
    
    if not _has_any(is_df, ['Operating Income', 'OperatingIncome']):
        missing.append('Operating Income')
    
    if not _has_any(is_df, ['Net Income', 'NetIncome', 'NetIncomeFromContinuingOperationNetMinorityInterest']):
        missing.append('Net Income')


    return missing


def build_aligned_for_ticker(ticker: str) -> pd.DataFrame:
    """Build an aligned dataset for one ticker."""

    bs, is_df, freq = load_statements(ticker, freq = TARGET_FREQ)
    missing = missing_required_columns(bs, is_df)
    
    if missing:

        print(f"{ticker}: skip, missing {missing}")

        return pd.DataFrame()

    period_days = 365.0 if freq == 'yearly' else 90.0
    growth_periods = 1 if freq == 'yearly' else 4

    features = compute_features(bs, is_df, days = period_days, growth_periods = growth_periods)
    targets = bs[TARGET_BS].copy()
    net_income = _pick(is_df, ['Net Income', 'NetIncome', 'NetIncomeFromContinuingOperationNetMinorityInterest']).rename(NET_INCOME_COL)
    net_income = net_income.reindex(targets.index)

    # Align features, targets, and net income on shared dates.
    dataset = features.join(targets, how = 'inner').join(net_income, how = 'inner')
    dataset = dataset.dropna()

    rev_series = _pick(is_df, ['Total Revenue', 'Operating Revenue', 'TotalRevenue', 'OperatingRevenue', 'Revenues'])
    # Shift state variables to create t-1 inputs.
    prev_state_df = bs.reindex(columns = STATE_COLS).shift(1)
    prev_state_df.columns = [f'prev_{c}' for c in STATE_COLS]
    prev_state_df['prev_Total Revenue'] = rev_series.reindex(targets.index).shift(1)

    # Combine current data with lagged state for modeling.
    aligned = dataset.join(prev_state_df, how = 'inner').dropna()
    aligned['ticker'] = ticker
    aligned['stmt_freq'] = freq


    return aligned


aligned_list = []

for ticker in [normalize_ticker(t) for t in TICKERS]:

    try:
        aligned_t = build_aligned_for_ticker(ticker)

        if aligned_t.empty:

            print(f"{ticker}: empty after alignment")

        else:

            print(f"{ticker}: aligned rows = {aligned_t.shape[0]}")
            aligned_list.append(aligned_t)

    except Exception as exc:

        print(f"{ticker}: failed ({exc})")

if not aligned_list:

    raise RuntimeError('No aligned data available')

aligned = pd.concat(aligned_list, axis = 0).sort_index()
prev_cols = [f'prev_{c}' for c in STATE_COLS] + ['prev_Total Revenue']
drop_cols = TARGET_BS + [NET_INCOME_COL] + prev_cols + ['ticker', 'stmt_freq']
FEATURE_COLS = [c for c in aligned.columns if c not in drop_cols]


print('Dataset shape:', aligned.shape)
print('Feature cols:', len(FEATURE_COLS), 'Target cols:', len(TARGET_LE) + 1)
print('Tickers:', aligned['ticker'].nunique())

AAPL: aligned rows = 3
GOOG: aligned rows = 2
0700.HK: aligned rows = 3
1810.HK: aligned rows = 3
IBM: aligned rows = 3
TSLA: aligned rows = 3
9633.HK: skip, missing ['Retained Earnings']
9633.HK: empty after alignment
9987.HK: aligned rows = 3
9988.HK: aligned rows = 2
IBKR: skip, missing ['Inventory', 'Net PPE']
IBKR: empty after alignment
KO: aligned rows = 3
MCD: aligned rows = 3
EL: aligned rows = 3
BRK-B: skip, missing ['Accounts Payable', 'Inventory', 'Cost Of Revenue', 'Operating Income']
BRK-B: empty after alignment
NESN.SW: aligned rows = 3
Dataset shape: (34, 25)
Feature cols: 11 Target cols: 3
Tickers: 12


### 2.3 Prev-state Matrix for Algebraic Layer

In [8]:
X_feat = aligned[FEATURE_COLS].values.astype('float32')
Y_bs = aligned[TARGET_LE].values.astype('float32')
Y_earn = aligned[[NET_INCOME_COL]].values.astype('float32')
X_prev = aligned[prev_cols].values.astype('float32')


print('Aligned shapes:', X_feat.shape, X_prev.shape, Y_bs.shape, Y_earn.shape)

Aligned shapes: (34, 11) (34, 8) (34, 2) (34, 1)


### 2.4 Scaling (z-score) for Stability

In [9]:
feat_scaler = StandardScaler()
bs_scaler = StandardScaler()
earn_scaler = StandardScaler()
prev_scaler = StandardScaler()

X_feat_scaled = feat_scaler.fit_transform(X_feat)
Y_bs_scaled = bs_scaler.fit_transform(Y_bs)
Y_earn_scaled = earn_scaler.fit_transform(Y_earn)
X_prev_scaled = prev_scaler.fit_transform(X_prev)

### 2.5 Train/Val Split on Scaled Data

In [10]:
n = X_feat_scaled.shape[0]
train_size = max(1, int(0.8 * n))

X_train_feat = X_feat_scaled[:train_size]
Y_train_bs = Y_bs_scaled[:train_size]
Y_train_earn = Y_earn_scaled[:train_size]
X_train_prev = X_prev_scaled[:train_size]

X_val_feat = X_feat_scaled[train_size:] if train_size < n else X_feat_scaled[train_size - 1:]
Y_val_bs = Y_bs_scaled[train_size:] if train_size < n else Y_bs_scaled[train_size - 1:]
Y_val_earn = Y_earn_scaled[train_size:] if train_size < n else Y_earn_scaled[train_size - 1:]
X_val_prev = X_prev_scaled[train_size:] if train_size < n else X_prev_scaled[train_size - 1:]


print('Scaled train:', X_train_feat.shape, Y_train_bs.shape, Y_train_earn.shape)
print('Scaled val:', X_val_feat.shape, Y_val_bs.shape, Y_val_earn.shape)

Scaled train: (27, 11) (27, 2) (27, 1)
Scaled val: (7, 11) (7, 2) (7, 1)


## 3. TensorFlow (Pareja/Pelaez Constrained)

### 3.1 TF Model with Algebraic Generator + Earnings Head

In [11]:
class AlgebraicBS(keras.layers.Layer):
    """Layer that enforces algebraic balance sheet constraints."""

    def __init__(self):
        """Initialize sublayers for balance sheet generation."""
        
        super().__init__()
        
        self.hidden = keras.layers.Dense(64, activation = 'relu')
        self.rev_head = keras.layers.Dense(1, activation = 'relu')
        self.cogs_head = keras.layers.Dense(1, activation = 'relu')
        self.drivers = keras.layers.Dense(5)
        self.margin_head = keras.layers.Dense(1)
        self.payout_head = keras.layers.Dense(1)
        self.earn_head = keras.layers.Dense(1, name = 'net_income_head')


    def call(self, inputs: Tuple[tf.Tensor, tf.Tensor]):
        """Compute constrained balance sheet and earnings outputs."""

        if not isinstance(inputs, (tuple, list)):

            raise TypeError("inputs must be (features, prev_state)")

        features, prev_state = inputs
        ar_prev, ap_prev, inv_prev, ppe_prev, liab_prev, equity_prev, re_prev, rev_prev = tf.split(prev_state, num_or_size_splits = 8, axis = -1)
        hidden = self.hidden(features)
        rev_predicate = self.rev_head(hidden)
        cogs_predicate = self.cogs_head(hidden)
        drivers_raw = self.drivers(hidden)

        dso = tf.nn.softplus(drivers_raw[:, 0:1])
        dpo = tf.nn.softplus(drivers_raw[:, 1:2])
        dih = tf.nn.softplus(drivers_raw[:, 2:3])
        dep_rate = tf.nn.sigmoid(drivers_raw[:, 3:4]) * 0.2
        capex_rate = tf.nn.sigmoid(drivers_raw[:, 4:5]) * 0.2
        net_margin = tf.tanh(self.margin_head(hidden)) * 0.5
        div_payout = tf.nn.sigmoid(self.payout_head(hidden))

        sales_per_day = rev_predicate / 365.0
        cogs_per_day = cogs_predicate / 365.0

        ar_next = dso * sales_per_day
        ap_next = dpo * cogs_per_day
        inv_next = dih * cogs_per_day
        dep = dep_rate * ppe_prev
        capex = capex_rate * rev_predicate
        ppe_next = ppe_prev + capex - dep
        net_income = net_margin * rev_predicate
        earn_predicate = self.earn_head(hidden)
        div = div_payout * net_income
        re_next = re_prev + net_income - div

        other_equity = tf.nn.relu(equity_prev - re_prev)

        equity_next = re_next + other_equity

        other_liab_prev = tf.nn.relu(liab_prev - ap_prev)
        growth = tf.where(rev_prev > 0, rev_predicate / rev_prev - 1.0, tf.zeros_like(rev_predicate))

        other_liab_next = other_liab_prev * (1.0 + growth)
        liab_next = ap_next + other_liab_next
        assets_wo_cash = ar_next + inv_next + ppe_next
        cash_next = equity_next + liab_next - assets_wo_cash
        assets_next = liab_next + equity_next
        bs_out = tf.concat([liab_next, equity_next], axis = -1)


        return bs_out, earn_predicate


def build_pareja_model(feat_dim: int, state_dim: int = 8):
    """Build a Pareja/Pelaez-constrained model."""

    feat_in = keras.Input(shape = (feat_dim,), name = 'features')
    state_in = keras.Input(shape = (state_dim,), name = 'prev_state')
    bs_out, earn_out = AlgebraicBS()([feat_in, state_in])
    
    
    return keras.Model([feat_in, state_in], [bs_out, earn_out], name = 'bs_pareja_style')


pareja_model = build_pareja_model(feat_dim = X_feat_scaled.shape[1], state_dim = X_prev_scaled.shape[1])
pareja_model.summary()

Model: "bs_pareja_style"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ features            │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prev_state          │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ algebraic_bs        │ [(None, 2),       │      1,418 │ features[0][0],   │
│ (AlgebraicBS)       │ (None, 1)]        │            │ prev_state[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,418 (5.54 KB)

 Trainable params: 1,418 (5.54 KB)

 Non-trainable params: 0 (0.00 B)

### 3.2 Tests / Sanity Checks

#### 3.2.1 Unit test

In [12]:
print('Running Unit test...')

try: 
    idx = pd.to_datetime(['2023-12-31', '2024-12-31'])

    sample_bs = pd.DataFrame(
        {
            'TotalAssets': [100.0, 120.0],
            'TotalLiabilities': [60.0, 70.0],
            'StockholdersEquity': [40.0, 50.0],
            'AccountsReceivable': [10.0, 12.0],
            'AccountsPayable': [8.0, 9.0],
            'Inventory': [5.0, 6.0],
            'NetPPE': [20.0, 22.0],
            'RetainedEarnings': [15.0, 18.0],
        },

        index = idx,
    )

    sample_is = pd.DataFrame(
        {
            'TotalRevenue': [80.0, 90.0],
            'CostOfRevenue': [30.0, 35.0],
            'OperatingIncome': [10.0, 12.0],
            'NetIncome': [8.0, 9.0],
        },

        index = idx,
    )

    canon = canonicalize_bs(sample_bs)

    assert 'Total Assets' in canon.columns, 'canonicalize_bs failed for Total Assets'
    assert 'Total Liabilities' in canon.columns, 'canonicalize_bs failed for Total Liabilities'
    assert 'Total Equity' in canon.columns, 'canonicalize_bs failed for Total Equity'

    resid = identity_residual(canon)
    assert np.allclose(resid.values, 0.0), 'identity_residual should be zero for balanced data'

    missing = missing_required_columns(sample_bs, sample_is)
    assert missing == [], f'missing_required_columns unexpected: {missing}'

    features_test = compute_features(sample_bs, sample_is, days = 365.0, growth_periods = 1)

    expected_cols = {
        'dso', 'dpo', 'dih', 'gross_margin', 'op_margin', 'net_margin',
        'rev_yoy', 'cogs_yoy', 'netinc_yoy', 'log_rev', 'log_assets'
    }

    assert expected_cols.issubset(set(features_test.columns)), 'compute_features missing expected columns'
    assert np.isfinite(features_test.to_numpy()).all(), 'compute_features produced non-finite values'

    print('Unit tests passed.')

except Exception as e:
    print('Unit tests failed:', e)

    raise

Running Unit test...
Unit tests passed.


#### 3.2.2 Integration test

In [13]:
print('Running Integration test...')

try:
    assert aligned.shape[0] > 0, 'aligned dataset is empty'
    assert X_feat.shape[0] == Y_bs.shape[0] == Y_earn.shape[0] == X_prev.shape[0], 'array length mismatch'

    test_batch = min(2, X_train_feat.shape[0])
    assert test_batch > 0, 'empty training batch'

    bs_hat, earn_hat = pareja_model([X_train_feat[:test_batch], X_train_prev[:test_batch]], training = False)
    assert bs_hat.shape[-1] == len(TARGET_LE), 'unexpected BS output width'
    assert earn_hat.shape[-1] == 1, 'unexpected earnings output width'

    print('Integration tests passed.')

except Exception as e:
    print('Integration tests failed:', e)

    raise


Running Integration test...
Integration tests passed.


### 3.3 Train/Evaluate (MAE on BS + Earnings)

In [14]:
pareja_model.compile(
    optimizer = keras.optimizers.Adam(1e-3),
    loss = [keras.losses.MeanAbsoluteError(), keras.losses.MeanAbsoluteError()],
    loss_weights = [1.0, 0.3],
)

hist = pareja_model.fit(
    [X_train_feat, X_train_prev], [Y_train_bs, Y_train_earn],
    
    validation_data = (
        [X_val_feat, X_val_prev], [Y_val_bs, Y_val_earn]
    ) if len(X_val_feat) > 0 else None,
    
    epochs = 20,
    batch_size = 2,
    verbose = 0,
)


print('Final Train Losses:', '\n', {k: v[-1] for k, v in hist.history.items() if 'loss' in k}, '\n')

bs_predicate_scaled, earn_predicate_scaled = pareja_model.predict([X_feat_scaled, X_prev_scaled], verbose = 0)
bs_predicate = bs_scaler.inverse_transform(bs_predicate_scaled)
earn_predicate = earn_scaler.inverse_transform(earn_predicate_scaled)

2026-01-27 16:56:18.282786: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_17}}


Final Train Losses: 
 {'algebraic_bs_loss': 0.057867713272571564, 'loss': 0.44238030910491943, 'val_algebraic_bs_loss': 0.28929322957992554, 'val_loss': 0.5173493027687073} 



In [15]:
bs_predicate_df = pd.DataFrame(bs_predicate, columns = TARGET_LE, index = aligned.index)
assets_predicate = bs_predicate_df['Total Liabilities'] + bs_predicate_df['Total Equity']
bs_predicate_full = bs_predicate_df.copy()
bs_predicate_full['Total Assets'] = assets_predicate
bs_predicate_full = bs_predicate_full[TARGET_BS]
resid_predicate = bs_predicate_full['Total Assets'] - (bs_predicate_full['Total Liabilities'] + bs_predicate_full['Total Equity'])

pred_full = bs_predicate_full.copy()
pred_full['ticker'] = aligned['ticker'].values
pred_full['stmt_freq'] = aligned['stmt_freq'].values
pred_full['pred_net_income'] = earn_predicate.flatten()
pred_full['pred_resid'] = pred_full['Total Assets'] - (pred_full['Total Liabilities'] + pred_full['Total Equity'])

actual_bs = aligned[TARGET_BS].copy()
actual_bs.columns = [f'actual_{c}' for c in actual_bs.columns]
pred_full = pd.concat([pred_full, actual_bs], axis = 1)
pred_full['actual_net_income'] = aligned[NET_INCOME_COL].values

pred_full['err_assets'] = pred_full['Total Assets'] - pred_full['actual_Total Assets']
pred_full['err_liab'] = pred_full['Total Liabilities'] - pred_full['actual_Total Liabilities']
pred_full['err_equity'] = pred_full['Total Equity'] - pred_full['actual_Total Equity']
pred_full['err_net_income'] = pred_full['pred_net_income'] - pred_full['actual_net_income']
pred_full['abs_err_assets'] = pred_full['err_assets'].abs()
pred_full['abs_err_liab'] = pred_full['err_liab'].abs()
pred_full['abs_err_equity'] = pred_full['err_equity'].abs()
pred_full['abs_err_net_income'] = pred_full['err_net_income'].abs()

summary = pred_full.groupby('ticker').agg(
    samples = ('pred_net_income', 'size'),
    mean_pred_assets = ('Total Assets', 'mean'),
    mean_pred_liab = ('Total Liabilities', 'mean'),
    mean_pred_equity = ('Total Equity', 'mean'),
    mean_pred_net_income = ('pred_net_income', 'mean'),
    mean_actual_assets = ('actual_Total Assets', 'mean'),
    mean_actual_liab = ('actual_Total Liabilities', 'mean'),
    mean_actual_equity = ('actual_Total Equity', 'mean'),
    mean_actual_net_income = ('actual_net_income', 'mean'),
    mae_assets = ('abs_err_assets', 'mean'),
    mae_liab = ('abs_err_liab', 'mean'),
    mae_equity = ('abs_err_equity', 'mean'),
    mae_net_income = ('abs_err_net_income', 'mean'),
)

def format_df_for_view(df: pd.DataFrame, decimals: int = 2) -> pd.DataFrame:
    """Create a display-only view with formatted numerics."""

    view = df.copy()
    num_cols = view.select_dtypes(include = [np.number]).columns
    fmt = f"{{:,.{decimals}f}}"
    view[num_cols] = view[num_cols].map(lambda v: "" if pd.isna(v) else fmt.format(v))

    return view


bs_predicate_full_view = format_df_for_view(bs_predicate_full)
pred_full_view = format_df_for_view(pred_full)
summary_view = format_df_for_view(summary)

earn_head_str = np.array2string(
    earn_predicate.flatten(),
    formatter = {"float_kind": lambda v: f"{v:,.2f}"}
)

In [16]:
print('Summary:', '\n')
summary_view

Summary: 



,samples,mean_pred_assets,mean_pred_liab,mean_pred_equity,mean_pred_net_income,mean_actual_assets,mean_actual_liab,mean_actual_equity,mean_actual_net_income,mae_assets,mae_liab,mae_equity,mae_net_income
ticker,,,,,,,,,,,,,
0700.HK,3.00,"1,288,782,348,288.00","420,556,079,104.00","868,226,170,880.00","138,858,676,224.00","1,645,457,333,333.33","741,978,333,333.33","903,479,000,000.00","165,844,000,000.00","356,675,028,736.00","321,422,232,384.00","38,854,464,618.67","27,580,089,600.00"
1810.HK,3.00,"338,128,633,856.00","186,000,605,184.00","152,128,012,288.00","16,676,213,760.00","333,636,646,333.33","167,839,997,333.33","165,796,649,000.00","14,535,776,333.33","51,945,568,093.33","36,519,344,672.00","16,132,717,394.67","3,605,827,256.00"
9987.HK,3.00,"198,108,921,856.00","188,145,008,640.00","9,963,913,216.00","-334,583,136.00","11,659,333,333.33","4,757,333,333.33","6,902,000,000.00","726,666,666.67","186,449,593,984.00","183,387,680,768.00","3,061,913,216.00","1,061,249,792.00"
9988.HK,2.00,"1,295,854,862,336.00","189,096,853,504.00","1,106,757,943,296.00","77,726,236,672.00","1,758,936,500,000.00","641,176,500,000.00","1,117,760,000,000.00","76,396,000,000.00","463,081,637,664.00","452,079,654,688.00","20,888,689,728.00","1,330,240,768.00"
AAPL,3.00,"170,300,162,048.00","188,035,579,904.00","-17,735,417,856.00","96,365,174,784.00","358,934,666,666.67","294,658,333,333.33","64,276,333,333.33","100,913,666,666.67","188,634,493,696.00","106,622,742,506.67","82,011,751,189.33","4,548,491,882.67"
EL,3.00,"196,093,493,248.00","193,915,568,128.00","2,177,919,744.00","-3,204,890,624.00","21,661,333,333.33","16,462,666,666.67","5,198,666,666.67","87,666,666.67","174,432,154,453.33","177,452,901,461.33","5,636,322,858.67","6,919,305,024.00"
GOOG,2.00,"452,753,424,384.00","185,760,071,680.00","266,993,336,320.00","66,219,225,088.00","383,828,000,000.00","114,066,500,000.00","269,761,500,000.00","66,883,500,000.00","68,925,408,000.00","71,693,571,680.00","11,266,600,800.00","4,490,325,728.00"
IBM,3.00,"448,740,294,656.00","273,936,105,472.00","174,804,156,416.00","6,842,824,704.00","133,219,666,666.67","109,210,666,666.67","24,009,000,000.00","5,055,000,000.00","315,520,595,221.33","164,725,433,344.00","150,795,159,146.67","3,771,843,733.33"
KO,3.00,"255,666,012,160.00","225,707,245,568.00","29,958,744,064.00","13,548,889,088.00","97,005,000,000.00","70,445,666,666.67","26,559,333,333.33","10,295,666,666.67","158,661,001,237.33","155,261,589,824.00","6,912,184,106.67","3,464,088,512.00"


In [17]:
print('Balance Sheet Predictions:', '\n')
bs_predicate_full_view.tail(100)

Balance Sheet Predictions: 



,Total Assets,Total Liabilities,Total Equity
2022-12-31,"295,644,659,712.00","240,434,610,176.00","55,210,049,536.00"
2022-12-31,"250,191,265,792.00","229,634,424,832.00","20,556,840,960.00"
2022-12-31,"244,393,410,560.00","185,758,236,672.00","58,635,182,080.00"
2022-12-31,"450,404,352,000.00","185,761,906,688.00","264,642,428,928.00"
2022-12-31,"1,085,425,188,864.00","297,162,702,848.00","788,262,486,016.00"
2022-12-31,"198,697,189,376.00","188,189,294,592.00","10,507,894,784.00"
2022-12-31,"223,793,692,672.00","185,758,236,672.00","38,035,456,000.00"
2022-12-31,"328,621,555,712.00","185,758,236,672.00","142,863,302,656.00"
2022-12-31,"478,690,476,032.00","277,218,295,808.00","201,472,180,224.00"
2023-03-31,"1,278,732,140,544.00","187,701,805,056.00","1,091,030,286,336.00"


In [18]:
print('Full Predictions:', '\n')
pred_full_view.tail(100)

Full Predictions: 



,Total Assets,Total Liabilities,Total Equity,ticker,stmt_freq,pred_net_income,pred_resid,actual_Total Assets,actual_Total Liabilities,actual_Total Equity,actual_net_income,err_assets,err_liab,err_equity,err_net_income,abs_err_assets,abs_err_liab,abs_err_equity,abs_err_net_income
2022-12-31,"295,644,659,712.00","240,434,610,176.00","55,210,049,536.00",MCD,yearly,"3,273,543,680.00",0.00,"50,435,600,000.00","56,439,000,000.00","-6,003,400,000.00","6,177,000,000.00","245,209,059,712.00","183,995,610,176.00","61,213,449,536.00","-2,903,456,320.00","245,209,059,712.00","183,995,610,176.00","61,213,449,536.00","2,903,456,320.00"
2022-12-31,"250,191,265,792.00","229,634,424,832.00","20,556,840,960.00",KO,yearly,"9,225,701,376.00",0.00,"92,763,000,000.00","66,937,000,000.00","25,826,000,000.00","9,542,000,000.00","157,428,265,792.00","162,697,424,832.00","-5,269,159,040.00","-316,298,624.00","157,428,265,792.00","162,697,424,832.00","5,269,159,040.00","316,298,624.00"
2022-12-31,"244,393,410,560.00","185,758,236,672.00","58,635,182,080.00",NESN.SW,yearly,"10,402,189,312.00",0.00,"135,182,000,000.00","92,390,000,000.00","42,792,000,000.00","9,270,000,000.00","109,211,410,560.00","93,368,236,672.00","15,843,182,080.00","1,132,189,312.00","109,211,410,560.00","93,368,236,672.00","15,843,182,080.00","1,132,189,312.00"
2022-12-31,"450,404,352,000.00","185,761,906,688.00","264,642,428,928.00",GOOG,yearly,"63,798,050,816.00",0.00,"365,264,000,000.00","109,120,000,000.00","256,144,000,000.00","59,972,000,000.00","85,140,352,000.00","76,641,906,688.00","8,498,428,928.00","3,826,050,816.00","85,140,352,000.00","76,641,906,688.00","8,498,428,928.00","3,826,050,816.00"
2022-12-31,"1,085,425,188,864.00","297,162,702,848.00","788,262,486,016.00",0700.HK,yearly,"162,303,148,032.00",0.00,"1,578,131,000,000.00","795,271,000,000.00","782,860,000,000.00","188,243,000,000.00","-492,705,811,136.00","-498,108,297,152.00","5,402,486,016.00","-25,939,851,968.00","492,705,811,136.00","498,108,297,152.00","5,402,486,016.00","25,939,851,968.00"
2022-12-31,"198,697,189,376.00","188,189,294,592.00","10,507,894,784.00",9987.HK,yearly,"-1,458,843,648.00",0.00,"11,826,000,000.00","4,666,000,000.00","7,160,000,000.00","442,000,000.00","186,871,189,376.00","183,523,294,592.00","3,347,894,784.00","-1,900,843,648.00","186,871,189,376.00","183,523,294,592.00","3,347,894,784.00","1,900,843,648.00"
2022-12-31,"223,793,692,672.00","185,758,236,672.00","38,035,456,000.00",TSLA,yearly,"16,450,433,024.00",0.00,"82,338,000,000.00","36,440,000,000.00","45,898,000,000.00","12,583,000,000.00","141,455,692,672.00","149,318,236,672.00","-7,862,544,000.00","3,867,433,024.00","141,455,692,672.00","149,318,236,672.00","7,862,544,000.00","3,867,433,024.00"
2022-12-31,"328,621,555,712.00","185,758,236,672.00","142,863,302,656.00",1810.HK,yearly,"3,626,856,448.00",0.00,"273,507,211,000.00","129,584,151,000.00","143,923,060,000.00","2,474,030,000.00","55,114,344,712.00","56,174,085,672.00","-1,059,757,344.00","1,152,826,448.00","55,114,344,712.00","56,174,085,672.00","1,059,757,344.00","1,152,826,448.00"
2022-12-31,"478,690,476,032.00","277,218,295,808.00","201,472,180,224.00",IBM,yearly,"1,867,137,024.00",0.00,"127,243,000,000.00","105,222,000,000.00","22,021,000,000.00","1,640,000,000.00","351,447,476,032.00","171,996,295,808.00","179,451,180,224.00","227,137,024.00","351,447,476,032.00","171,996,295,808.00","179,451,180,224.00","227,137,024.00"
2023-03-31,"1,278,732,140,544.00","187,701,805,056.00","1,091,030,286,336.00",9988.HK,yearly,"74,767,540,224.00",0.00,"1,753,044,000,000.00","630,123,000,000.00","1,122,921,000,000.00","72,783,000,000.00","-474,311,859,456.00","-442,421,194,944.00","-31,890,713,664.00","1,984,540,224.00","474,311,859,456.00","442,421,194,944.00","31,890,713,664.00","1,984,540,224.00"
